In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
! pip install stemming

# 第6章: 英語テキストの処理
英語のテキスト（nlp.txt）に対して，以下の処理を実行せよ．

## 50. 文区切り
(. or ; or : or ? or !) → 空白文字 → 英大文字というパターンを文の区切りと見なし，入力された文書を1行1文の形式で出力せよ．

In [2]:
import re

nlp_file = "/content/drive/My Drive/Colab Notebooks/Python100本ノック/nlp.txt"
with open(nlp_file) as f:
    data = f.read()
    for line in re.split('[.;:?!] [A-Z]', data):
        print(line)

Natural language processing
From Wikipedia, the free encyclopedia

Natural language processing (NLP) is a field of computer science, artificial intelligence, and linguistics concerned with the interactions between computers and human (natural) languages
s such, NLP is related to the area of humani-computer interaction
any challenges in NLP involve natural language understanding, that is, enabling computers to derive meaning from human or natural language input, and others involve natural language generation.

History

The history of NLP generally starts in the 1950s, although work can be found from earlier periods
n 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence.

The Georgetown experiment in 1954 involved fully automatic translation of more than sixty Russian sentences into English
he authors claimed that within three or five years, machine translation would be a solv

## 51. 単語の切り出し
空白を単語の区切りとみなし，50の出力を入力として受け取り，1行1単語の形式で出力せよ．ただし，文の終端では空行を出力せよ．

In [4]:
import re

nlp_file = "/content/drive/My Drive/Colab Notebooks/Python100本ノック/nlp.txt"
with open(nlp_file) as f:
    data = f.read()
    for line in re.split('[.;:?!] [A-Z]', data):
        for word in line.split(" "):
            print(word)
        print("\n")
        # 長いので1行分だけ出力
        break

Natural
language
processing
From
Wikipedia,
the
free
encyclopedia

Natural
language
processing
(NLP)
is
a
field
of
computer
science,
artificial
intelligence,
and
linguistics
concerned
with
the
interactions
between
computers
and
human
(natural)
languages




## 52. ステミング
51の出力を入力として受け取り，Porterのステミングアルゴリズムを適用し，単語と語幹をタブ区切り形式で出力せよ． Pythonでは，Porterのステミングアルゴリズムの実装としてstemmingモジュールを利用するとよい．

In [7]:
from nltk.stem.porter import PorterStemmer as ps

import re

nlp_file = "/content/drive/My Drive/Colab Notebooks/Python100本ノック/nlp.txt"
with open(nlp_file) as f:
    data = f.read()
    porter = ps()
    for line in re.split('[.;:?!] [A-Z]', data):
        for word in line.split(" "):
            print("{}\t{}".format(word, porter.stem(word)))
        print("\n")
        # 長いので1行分だけ出力
        break

Natural	natur
language	languag
processing
From	processing
from
Wikipedia,	wikipedia,
the	the
free	free
encyclopedia

Natural	encyclopedia

natur
language	languag
processing	process
(NLP)	(nlp)
is	is
a	a
field	field
of	of
computer	comput
science,	science,
artificial	artifici
intelligence,	intelligence,
and	and
linguistics	linguist
concerned	concern
with	with
the	the
interactions	interact
between	between
computers	comput
and	and
human	human
(natural)	(natural)
languages	languag




## 53. Tokenization
Stanford Core NLPを用い，入力テキストの解析結果をXML形式で得よ．また，このXMLファイルを読み込み，入力テキストを1行1単語の形式で出力せよ．

In [0]:
!apt install openjdk-8-jre
!java --version

In [0]:
URL="http://nlp.stanford.edu/software/stanford-corenlp-full-2013-06-20.zip"
FILE_NAME="stanford-corenlp-full-2013-06-20.zip"
! curl -L -O $URL
! unzip $FILE_NAME -d /usr/local/lib/
! pip install corenlp-python 

In [0]:
!echo "annotators = tokenize, ssplit" > user.properties
# https://qiita.com/yubessy/items/1869ac2c66f4e76cd6c5
!cp user.properties /usr/local/lib/stanford-corenlp-full-2013-06-20/

In [24]:
# XMLの作成
!java -cp "/usr/local/lib/stanford-corenlp-full-2013-06-20/*" -Xmx3g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators tokenize,ssplit -file /content/nlp.txt

Adding annotator tokenize
Adding annotator ssplit

Processing file /content/nlp.txt ... writing to /content/nlp.txt.xml [0.214 seconds]
Annotation pipeline timing information:
PTBTokenizerAnnotator: 0.1 sec.
WordsToSentencesAnnotator: 0.0 sec.
TOTAL: 0.1 sec. for 1452 tokens at 10755.6 tokens/sec.
Pipeline setup: 0.1 sec.
Total time for StanfordCoreNLP pipeline: 0.5 sec.


In [23]:
import json
import corenlp

# パーサの生成
corenlp_dir = "/usr/local/lib/stanford-corenlp-full-2013-06-20/"
properties_file = "./user.properties" 
parser = corenlp.StanfordCoreNLP(corenlp_path=corenlp_dir, properties=properties_file)

# パースして結果をpretty print
#result_json = json.loads(parser.parse("I am Alice."))
result_json = parser.parse("I am Alice.")
print(result_json)

{"sentences": [{"words": [["I", {"CharacterOffsetBegin": "0", "CharacterOffsetEnd": "1"}], ["am", {"CharacterOffsetBegin": "2", "CharacterOffsetEnd": "4"}], ["Alice", {"CharacterOffsetBegin": "5", "CharacterOffsetEnd": "10"}], [".", {"CharacterOffsetBegin": "10", "CharacterOffsetEnd": "11"}]], "parsetree": [], "dependencies": [], "indexeddependencies": [], "text": "I am Alice."}]}


In [27]:
import xml.etree.ElementTree as et

root = et.parse('nlp.txt.xml')
for ind, word in enumerate(root.iter('word')):
    if ind > 4:
        break
    # 多いので5行分だけ出力
    print(word.text)

Natural
language
processing
From
Wikipedia


## 54. 品詞タグ付け
Stanford Core NLPの解析結果XMLを読み込み，単語，レンマ，品詞をタブ区切り形式で出力せよ．

In [0]:
!echo "annotators = tokenize, ssplit, pos, lemma" > user.properties
# https://qiita.com/yubessy/items/1869ac2c66f4e76cd6c5
!cp user.properties /usr/local/lib/stanford-corenlp-full-2013-06-20/

In [29]:
# XMLの作成
!java -cp "/usr/local/lib/stanford-corenlp-full-2013-06-20/*" -Xmx3g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators tokenize,ssplit,pos,lemma -file /content/nlp.txt

Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Reading POS tagger model from edu/stanford/nlp/models/pos-tagger/english-left3words/english-left3words-distsim.tagger ... done [2.4 sec].
Adding annotator lemma

Processing file /content/nlp.txt ... writing to /content/nlp.txt.xml [0.179 seconds]
Annotation pipeline timing information:
PTBTokenizerAnnotator: 0.2 sec.
WordsToSentencesAnnotator: 0.0 sec.
POSTaggerAnnotator: 0.2 sec.
MorphaAnnotator: 0.1 sec.
TOTAL: 0.5 sec. for 1452 tokens at 2709.0 tokens/sec.
Pipeline setup: 2.4 sec.
Total time for StanfordCoreNLP pipeline: 3.2 sec.


In [34]:
import xml.etree.ElementTree as et

root = et.parse('nlp.txt.xml')
for word, lemma, pos in zip(root.iter('word'), root.iter("lemma"), root.iter("POS")):
    print("{}\t{}\t{}".format(word.text, lemma.text, pos.text))
    # 量が多いので1単語分のみ
    break

Natural	natural	JJ


## 55. 固有表現抽出
入力文中の人名をすべて抜き出せ．

In [0]:
!echo "annotators = tokenize, ssplit, pos, lemma, ner" > user.properties
# https://qiita.com/yubessy/items/1869ac2c66f4e76cd6c5
!cp user.properties /usr/local/lib/stanford-corenlp-full-2013-06-20/

In [50]:
# XMLの作成
!java -cp "/usr/local/lib/stanford-corenlp-full-2013-06-20/*" -Xmx3g edu.stanford.nlp.pipeline.StanfordCoreNLP \
  -annotators tokenize,ssplit,pos,lemma,ner -ner.useSUTime 0 -file /content/nlp.txt

Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Reading POS tagger model from edu/stanford/nlp/models/pos-tagger/english-left3words/english-left3words-distsim.tagger ... done [2.2 sec].
Adding annotator lemma
Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [3.7 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [2.5 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.conll.4class.distsim.crf.ser.gz ... done [2.3 sec].

Processing file /content/nlp.txt ... writing to /content/nlp.txt.xml [0.176 seconds]
Annotation pipeline timing information:
PTBTokenizerAnnotator: 0.1 sec.
WordsToSentencesAnnotator: 0.0 sec.
POSTaggerAnnotator: 0.2 sec.
MorphaAnnotator: 0.1 sec.
NERCombinerAnnotator: 1.5 sec.
TOTAL: 1.9 sec. for 1452 tokens at 757.0 tokens/sec.
Pipeline setup: 11.0 sec.
Total time for StanfordCoreNLP pipeline: 13.1 sec.


In [58]:
import xml.etree.ElementTree as et

root = et.parse('nlp.txt.xml')
for ner, word in zip(root.iter('NER'), root.iter("word")):
    # 量が多いので1単語分のみ
    if ner.text == "PERSON":
        print(word.text)

Alan
Turing
ELIZA
Joseph
Weizenbaum
Wilensky
Meehan
Lehnert
Carbonell
Lehnert
Jabberwacky
Moore


## 56. 共参照解析
Stanford Core NLPの共参照解析の結果に基づき，文中の参照表現（mention）を代表参照表現（representative mention）に置換せよ．ただし，置換するときは，「代表参照表現（参照表現）」のように，元の参照表現が分かるように配慮せよ．

In [0]:
!echo "annotators = tokenize, ssplit, pos, lemma, ner, parse, dcoref" > user.properties
# https://qiita.com/yubessy/items/1869ac2c66f4e76cd6c5
!cp user.properties /usr/local/lib/stanford-corenlp-full-2013-06-20/

In [71]:
# XMLの作成
!java -cp "/usr/local/lib/stanford-corenlp-full-2016-10-31/*" -Xmx5g edu.stanford.nlp.pipeline.StanfordCoreNLP \
  -annotators tokenize,ssplit,pos,lemma,ner,parse,dcoref -ner.useSUTime 0 -file /content/nlp.txt

[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator pos
[main] INFO edu.stanford.nlp.tagger.maxent.MaxentTagger - Loading POS tagger from edu/stanford/nlp/models/pos-tagger/english-left3words/english-left3words-distsim.tagger ... done [2.2 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
[main] INFO edu.stanford.nlp.ie.AbstractSequenceClassifier - Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [2.5 sec].
[main] INFO edu.stanford.nlp.ie.AbstractSequenceClassifier - Loading classifier from edu/stanford/nlp/models/ner/english.muc.7cla

In [0]:
import xml.etree.ElementTree as et

root = et.parse('nlp.txt.xml')
for coref in root.iter('coreferene'):
    mention = coref.findall("mention")
    represent = coref.find("mention[@representative="true"]")
#for ner, word in zip(root.iter('NER'), root.iter("word")):
#    # 量が多いので1単語分のみ
#    if ner.text == "PERSON":
#        print(word.text)

In [0]:
from nltk.stem.porter import PorterStemmer as ps

import re

nlp_file = "/content/drive/My Drive/Colab Notebooks/Python100本ノック/nlp.txt"
with open(nlp_file) as f:
    data = f.read()
    porter = ps()
    for line in re.split('[.;:?!] [A-Z]', data):
        for word in line.split(" "):
            print("{}\t{}".format(word, porter.stem(word)))
        print("\n")
        # 長いので1行分だけ出力
        break